TODO: change to use own model instead of pretrained

In [2]:
from pathlib import Path
import yaml
from pyannote.audio import Pipeline, Model
import os
from dotenv import load_dotenv
import re
from pydub import AudioSegment
from pyannote.audio.pipelines import SpeakerDiarization
from natsort import natsorted

load_dotenv()

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

C:\Users\verho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\verho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


In [3]:
trained_pipeline = Pipeline.from_pretrained("./pipeline/config.yaml")
trained_pipeline
model = Model.from_pretrained("./models/trained_model.ckpt")
trained_pipeline.segmentation_model = model

with open("./parameters/config.yaml", "r") as f:
    parameters = yaml.safe_load(f)

params = parameters["params"]

params_clust = params["clustering"]

trained_pipeline.instantiate({
        "segmentation": {"min_duration_off": params["segmentation"]["min_duration_off"]},
        "clustering": params["clustering"]
})

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
C:\Users\verho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pytorch_lightning\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.5.1.post0, which is newer than your current Lightning version: v2.5.0.post0


Model was trained with torch 2.7.0+cu126, yours is 2.6.0+cpu. This should be OK but you might want to upgrade torch.


In [4]:
pipeline = SpeakerDiarization(
    segmentation=model,
    embedding=trained_pipeline.embedding,
    embedding_exclude_overlap=trained_pipeline.embedding_exclude_overlap,
    clustering=trained_pipeline.klustering,
)

with open("./parameters/config.yaml", "r") as f:
    loaded_parameters = yaml.safe_load(f)

loaded_params = loaded_parameters["params"]

params_clust = loaded_params["clustering"]

parameters_segm = parameters["params"]["segmentation"]


pipeline.instantiate({
    "segmentation": {
        "min_duration_off": parameters_segm["min_duration_off"],
        "threshold": parameters_segm["threshold"]
    },
    "clustering": params_clust
})

In [5]:
# TODO: niet hardgecodeerd voor eindresultaat
path = "../training_database/wavs/1-03_student1_jongere_Lab1.wav"

In [7]:
for file in os.listdir("../training_database/wavs/geen_ruis"):
    print(file)
    %run ./audio_preprocessing.py "../training_database/wavs/geen_ruis/$file"

1-03_student1_jongere_Lab1.wav
../training_database/wavs/geen_ruis/1-03_student1_jongere_Lab1.wav
{'normal': {'original_length': 172.7, 'filtered_length': 172.7, 'silence_length': 0.0, 'silence_percentage': 0.0}, 'elder': {'original_length': 172.7, 'filtered_length': 172.7, 'silence_length': 0.0, 'silence_percentage': 0.0}}
1-03_student1_oudere_Lab2.wav
../training_database/wavs/geen_ruis/1-03_student1_oudere_Lab2.wav
{'normal': {'original_length': 100.5, 'filtered_length': 100.5, 'silence_length': 0.0, 'silence_percentage': 0.0}, 'elder': {'original_length': 100.5, 'filtered_length': 100.5, 'silence_length': 0.0, 'silence_percentage': 0.0}}
1-03_student2_jongere_Lab1.wav
../training_database/wavs/geen_ruis/1-03_student2_jongere_Lab1.wav
{'normal': {'original_length': 161.9, 'filtered_length': 161.9, 'silence_length': 0.0, 'silence_percentage': 0.0}, 'elder': {'original_length': 161.9, 'filtered_length': 161.9, 'silence_length': 0.0, 'silence_percentage': 0.0}}
1-03_student2_oudere_Lab

In [30]:
# apply pretrained pipeline
diarization = pipeline(path)    

/home/lisa_verhoeyen/.local/lib/python3.10/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


create rttm file based on model result

In [31]:
file_name = re.search(r'.*\/(.+?)\.wav', path).group(1)

# delete file if it already exists so a new file can get created
if os.path.exists(f"./results/rttm/{file_name}.rttm"):
    os.remove(f"./results/rttm/{file_name}.rttm")

f = open(f"./results/rttm/{file_name}.rttm", "w")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    f.write(f"SPEAKER {file_name} 1 {round(turn.start, 3)} {round(turn.end-turn.start, 3)} <NA> <NA> {speaker} <NA> <NA>\n")

f.close()

turn result into audio files

In [ ]:
audio = AudioSegment.from_wav(path)

os.mkdir(f"./results/audio/{file_name}/")


for turn, _, speaker in diarization.itertracks(yield_label=True):
    if not os.path.isdir(f"./results/audio/{file_name}/{speaker}"):
        os.mkdir(f"./results/audio/{file_name}/{speaker}")

    segment = audio[turn.start * 1000:turn.end * 1000]  # Convert seconds to milliseconds
    segment.export(f"./results/audio/{file_name}/{speaker}/speaker_{speaker}_{turn.start:.1f}-{turn.end:.1f}.wav", format="wav")

create audio file based on speaker by combining audio files

In [34]:
speakers = os.listdir(f"./results/audio/{file_name}")

for speaker in speakers:
    files_list = os.listdir(f"./results/audio/{file_name}/{speaker}")
    files_list = natsorted(files_list)

    combined = AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[0]}")
    for i in range(1,len(files_list)):
        combined = combined.append(AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[i]}"), crossfade=0.0)

    combined.export(f"./results/audio/{file_name}_{speaker}.wav", format="wav")